In [ ]:
import torch
import numpy as numpy
import torch.nn as nn
import torch.optim as optim

torch.set_printoptions(sci_mode=False)

def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()
    
def training_loop(n_epochs, optimizer, model, loss_fn, t_u_train, t_u_val, t_c_train, t_c_val):
    for epoch in range(1, n_epochs + 1):
        t_p_train = model(t_u_train)
        loss_train = loss_fn(t_p_train, t_c_train)

        t_p_val = model(t_u_val)
        loss_val = loss_fn(t_p_val, t_c_val)

        optimizer.zero_grad()
        loss_train.backward()
        optimizer.step()

        if epoch == 1 or epoch % 1000 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                  f" Validation loss {loss_val.item():.4f}")
    

t_c = torch.tensor([0.5,  14.0, 15.0, 28.0, 11.0,  8.0,  3.0, -4.0,  6.0, 13.0, 21.0]).unsqueeze(1)
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]).unsqueeze(1)
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

linear_model = nn.Linear(1, 1)
optimizer = optim.SGD(linear_model.parameters(), lr = 1e-2)

training_loop(
    n_epochs = 30000,
    optimizer = optimizer,
    model = linear_model,
    loss_fn = nn.MSELoss(),
    t_u_train = t_un_train,
    t_u_val = t_un_val,
    t_c_train = t_c_train,
    t_c_val = t_c_val)
print()
print(linear_model.weight)
print(linear_model.bias)


In [18]:
seq_model = nn.Sequential(
    nn.Linear(1, 13),
    nn.Tanh(),
    nn.Linear(13, 1)
)

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)
training_loop(
            n_epochs = 5000,
            optimizer = optimizer,
            model = seq_model,
            loss_fn = nn.MSELoss(),
            t_u_train = t_un_train,
            t_u_val = t_un_val,
            t_c_train = t_c_train,
            t_c_val = t_c_val)
print('output', seq_model(t_un_val))
print('answer', t_c_val)


Epoch 1, Training loss 196.5303, Validation loss 117.9417
Epoch 1000, Training loss 4.1602, Validation loss 0.6752
Epoch 2000, Training loss 3.1459, Validation loss 3.5027
Epoch 3000, Training loss 2.3044, Validation loss 3.6753
Epoch 4000, Training loss 2.0194, Validation loss 3.9578
Epoch 5000, Training loss 1.9259, Validation loss 4.2347
output tensor([[13.5381],
        [-2.1685]], grad_fn=<AddmmBackward0>)
answer tensor([[15.],
        [-4.]])
